# UAS PPW LDA berita

Link streamlit : https://uas-ppw-4csi75jojtcgfkwu4dr83s.streamlit.app

In [1]:
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pickle

In [2]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.svm import SVC

import pandas as pd
import warnings
import joblib
import nltk
import re

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/ppw/beritajatim.csv')
df

,Judul,Isi,Tanggal,Kategori
0,Pelatih Ungkap Dua Faktor Kekalahan Persema da...,Jember (beritajatim.com)– Persema Malang dihaj...,"Selasa, 5 Desember 2023, 21:52 WIB",Sport
1,"Jumpa Persebaya, Persis Belum Tentu Dipimpin L...",Jakarta (beritajatim.com)– Persis Solo belum t...,"Senin, 11 Desember 2023, 09:56 WIB",Sport
2,Eits! 229 Orang di Jombang Positif HIV/AIDS,Jombang (beritajatim.com) –Sebanyak 229 orang ...,"Minggu, 3 Desember 2023, 07:32 WIB",Pendidikan
3,Bupati Sidoarjo Siagakan BPBD Antisipasi Benca...,Sidoarjo (beritajatim.com)– Bupati Sidoarjo H....,"Rabu, 13 Desember 2023, 14:38 WIB",Politik
4,"Pesta Siaga ASIK 2023, Bentuk Generasi Berkara...",Lamongan (beritajatim.com) –Pramuka Lamongan m...,"Selasa, 21 November 2023, 21:54 WIB",Pendidikan
...,...,...,...,...
1095,PKS Tuban Gelar Kampanye Perdana dengan Flashmob,Tuban (beritajatim.com) –DPD Partai Keadilan S...,"Minggu, 10 Desember 2023, 18:46 WIB",Politik
1096,"Musim Pancaroba, Peternak Kediri Waspada Penya...",Kediri (beritajatim.com) –Memasuki musim panca...,"Selasa, 14 November 2023, 09:10 WIB",Pendidikan
1097,"Dukung Ganjar, Pemulung di Blitar Ikut Nobar D...",Blitar (beritajatim.com) –Seorang pemulung di ...,"Selasa, 12 Desember 2023, 21:48 WIB",Politik
1098,"Lawan Simo Putra, PSMP Target Kembali Raih Poin",Mojokerto (beritajatim.com)– Tim PS Mojokerto ...,"Kamis, 7 Desember 2023, 11:15 WIB",Sport


In [5]:
pd.crosstab(index=df['Kategori'], columns='count')

col_0,count
Kategori,
Pendidikan,400
Politik,300
Sport,400


In [6]:
df.isnull().sum()

Judul       0
Isi         3
Tanggal     0
Kategori    0
dtype: int64

In [7]:
def cleaning(text):
    if not isinstance(text, str):
        return str(text)
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text).strip()
    return cleaned_text

# Apply the updated cleaning function to handle non-string values
df['Cleaning'] = df['Isi'].apply(cleaning)
df['Cleaning']

0       Jember beritajatimcom Persema Malang dihajar  ...
1       Jakarta beritajatimcom Persis Solo belum tentu...
2       Jombang beritajatimcom Sebanyak  orang di Jomb...
3       Sidoarjo beritajatimcom Bupati Sidoarjo H Ahma...
4       Lamongan beritajatimcom Pramuka Lamongan mengg...
                              ...                        
1095    Tuban beritajatimcom DPD Partai Keadilan Sejah...
1096    Kediri beritajatimcom Memasuki musim pancaroba...
1097    Blitar beritajatimcom Seorang pemulung di Kabu...
1098    Mojokerto beritajatimcom Tim PS Mojokerto Putr...
1099    Banyuwangi beritajatimcom Atlet Banyuwangi Ald...
Name: Cleaning, Length: 1100, dtype: object

In [8]:
def tokenizer(text):
  text = text.lower()
  return word_tokenize(text)

df['Tokenizing'] = df['Cleaning'].apply(tokenizer)
df['Tokenizing']

0       [jember, beritajatimcom, persema, malang, diha...
1       [jakarta, beritajatimcom, persis, solo, belum,...
2       [jombang, beritajatimcom, sebanyak, orang, di,...
3       [sidoarjo, beritajatimcom, bupati, sidoarjo, h...
4       [lamongan, beritajatimcom, pramuka, lamongan, ...
                              ...                        
1095    [tuban, beritajatimcom, dpd, partai, keadilan,...
1096    [kediri, beritajatimcom, memasuki, musim, panc...
1097    [blitar, beritajatimcom, seorang, pemulung, di...
1098    [mojokerto, beritajatimcom, tim, ps, mojokerto...
1099    [banyuwangi, beritajatimcom, atlet, banyuwangi...
Name: Tokenizing, Length: 1100, dtype: object

In [9]:
corpus = stopwords.words('indonesian')

def stopwordText(words):
 return [word for word in words if word not in corpus]

df['Stopword Removal'] = df['Tokenizing'].apply(stopwordText)

# Gabungkan kembali token menjadi kalimat utuh
df['Full Text'] = df['Stopword Removal'].apply(lambda x: ' '.join(x))
df['Full Text']

0       jember beritajatimcom persema malang dihajar t...
1       jakarta beritajatimcom persis solo dipimpin sa...
2       jombang beritajatimcom orang jombang positif t...
3       sidoarjo beritajatimcom bupati sidoarjo h ahma...
4       lamongan beritajatimcom pramuka lamongan mengg...
                              ...                        
1095    tuban beritajatimcom dpd partai keadilan sejah...
1096    kediri beritajatimcom memasuki musim pancaroba...
1097    blitar beritajatimcom pemulung kabupaten blita...
1098    mojokerto beritajatimcom tim ps mojokerto putr...
1099    banyuwangi beritajatimcom atlet banyuwangi ald...
Name: Full Text, Length: 1100, dtype: object

In [10]:
def tfidf(dokumen, kategori):
  vectorizer = TfidfVectorizer()
  x = vectorizer.fit_transform(dokumen).toarray()
  terms = vectorizer.get_feature_names_out()

  final_tfidf = pd.DataFrame(x, columns=terms)
  final_tfidf.insert(0, 'Dokumen', dokumen)
  final_tfidf.insert(len(final_tfidf.columns),'Kategori', kategori)

  return (vectorizer, final_tfidf)

tfidf_vectorizer, final_tfidf = tfidf(df['Full Text'], df['Kategori'])
final_tfidf

,Dokumen,aa,aam,aamsuf,aan,aang,abaca,abad,abadi,abadikan,...,zulfikri,zulhafmi,zulham,zulhas,zulhaszulhas,zulkifli,zulverdi,zunaich,zunaih,Kategori
0,jember beritajatimcom persema malang dihajar t...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sport
1,jakarta beritajatimcom persis solo dipimpin sa...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sport
2,jombang beritajatimcom orang jombang positif t...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pendidikan
3,sidoarjo beritajatimcom bupati sidoarjo h ahma...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Politik
4,lamongan beritajatimcom pramuka lamongan mengg...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pendidikan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,tuban beritajatimcom dpd partai keadilan sejah...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Politik
1096,kediri beritajatimcom memasuki musim pancaroba...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pendidikan
1097,blitar beritajatimcom pemulung kabupaten blita...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Politik
1098,mojokerto beritajatimcom tim ps mojokerto putr...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sport


In [11]:
X = final_tfidf.drop(['Dokumen', 'Kategori'], axis=1)
X

,aa,aam,aamsuf,aan,aang,abaca,abad,abadi,abadikan,abai,...,zulfikarzulfikar,zulfikri,zulhafmi,zulham,zulhas,zulhaszulhas,zulkifli,zulverdi,zunaich,zunaih
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
y = df['Kategori']
y

0            Sport
1            Sport
2       Pendidikan
3          Politik
4       Pendidikan
           ...    
1095       Politik
1096    Pendidikan
1097       Politik
1098         Sport
1099       Politik
Name: Kategori, Length: 1100, dtype: object

## Splitting Data

Dalam proses ini menggunakan modul `train_test_split` dari Scikit-Learn untuk membagi dataset menjadi subset pelatihan (training) dan pengujian (testing). Berikut penjelasan singkat tentang apa yang terjadi:

- `X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)`: Ini adalah perintah yang membagi data menjadi empat subset, yaitu `X_train` (fitur pelatihan), `X_test` (fitur pengujian), `y_train` (target pelatihan), dan `y_test` (target pengujian).

    - `X` adalah DataFrame yang berisi fitur.
    - `y` adalah Series yang berisi target atau label.
    - `test_size=0.3` menentukan bahwa 30% dari data akan digunakan sebagai data pengujian, sementara 70% akan digunakan sebagai data pelatihan.
    - `random_state=42` digunakan untuk mengatur nilai seed agar pembagian data dapat direproduksi dengan hasil yang konsisten. Anda dapat menggantinya dengan nilai lain jika diperlukan.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## **Modeling With LDA**

### Mencari Best Parameter untuk LDA

In [14]:
def find_best_lda(Xtrain, Xtest, n_components, alpha, beta):
  looping = 1
  best = {'k' : 0, 'alpha' : 0, 'beta' : 0, 'accuracy' : 0, 'model': '', 'lda' : '', 'lda_Xtrain' : '', 'lda_Xtest' : ''}
  history = pd.DataFrame(columns=["Pengujian Ke", "K", "Alpha", "Beta", "Accuracy"])

  # Menambahkan tqdm pada loop terluar
  for k in n_components:
    for a in alpha:
      for b in beta:
        lda = LatentDirichletAllocation(n_components=k, doc_topic_prior=a, topic_word_prior=b)
        lda_Xtrain = lda.fit_transform(Xtrain)
        lda_Xtest = lda.transform(Xtest)

        # Membuat model Naive Bayes
        model = GaussianNB()

        # Melatih model pada data pelatihan
        model.fit(lda_Xtrain, y_train)

        # Melakukan prediksi pada data pengujian
        y_pred = model.predict(lda_Xtest)

        # Menghitung akurasi
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Jumlah Topik: {k}, Alpha: {a}, Beta: {b}, Accuracy: {accuracy}")

        if accuracy > best['accuracy']:
          best['accuracy'] = accuracy
          best['k'] = k
          best['alpha'] = a
          best['beta'] = b
          best['model'] = model
          best['lda'] = lda
          best['lda_Xtrain'] = lda_Xtrain
          best['lda_Xtest'] = lda_Xtest

        history.loc[len(history)] = [f"Pengujian Ke- {looping}", k, a, b, accuracy]
        looping += 1

  return (best, history)

k = [3, 4]
alpha = [0.1, 0.2]
beta = [0.1, 0.2]
best_param, history = find_best_lda(X_train, X_test, k, alpha, beta)

Jumlah Topik: 3, Alpha: 0.1, Beta: 0.1, Accuracy: 0.4090909090909091
Jumlah Topik: 3, Alpha: 0.1, Beta: 0.2, Accuracy: 0.6212121212121212
Jumlah Topik: 3, Alpha: 0.2, Beta: 0.1, Accuracy: 0.4636363636363636
Jumlah Topik: 3, Alpha: 0.2, Beta: 0.2, Accuracy: 0.48484848484848486
Jumlah Topik: 4, Alpha: 0.1, Beta: 0.1, Accuracy: 0.4696969696969697
Jumlah Topik: 4, Alpha: 0.1, Beta: 0.2, Accuracy: 0.49696969696969695
Jumlah Topik: 4, Alpha: 0.2, Beta: 0.1, Accuracy: 0.42727272727272725
Jumlah Topik: 4, Alpha: 0.2, Beta: 0.2, Accuracy: 0.3878787878787879


In [15]:
best_param

{'k': 3,
 'alpha': 0.1,
 'beta': 0.2,
 'accuracy': 0.6212121212121212,
 'model': GaussianNB(),
 'lda': LatentDirichletAllocation(doc_topic_prior=0.1, n_components=3,
                           topic_word_prior=0.2),
 'lda_Xtrain': array([[0.97544701, 0.0122783 , 0.01227468],
        [0.01216281, 0.97568042, 0.01215676],
        [0.0104039 , 0.97919636, 0.01039975],
        ...,
        [0.01266082, 0.97467943, 0.01265975],
        [0.01230141, 0.97540073, 0.01229787],
        [0.97942335, 0.0102896 , 0.01028705]]),
 'lda_Xtest': array([[0.9764094 , 0.01179657, 0.01179404],
        [0.01208824, 0.97582605, 0.01208571],
        [0.01143833, 0.97712402, 0.01143765],
        [0.4370294 , 0.55202101, 0.0109496 ],
        [0.01300327, 0.97399181, 0.01300492],
        [0.01157503, 0.97685367, 0.0115713 ],
        [0.97880996, 0.01059637, 0.01059367],
        [0.97657866, 0.01171214, 0.01170919],
        [0.0121609 , 0.9756796 , 0.0121595 ],
        [0.0135461 , 0.97290883, 0.01354507],
      

In [16]:
history

,Pengujian Ke,K,Alpha,Beta,Accuracy
0,Pengujian Ke- 1,3,0.1,0.1,0.409091
1,Pengujian Ke- 2,3,0.1,0.2,0.621212
2,Pengujian Ke- 3,3,0.2,0.1,0.463636
3,Pengujian Ke- 4,3,0.2,0.2,0.484848
4,Pengujian Ke- 5,4,0.1,0.1,0.469697
5,Pengujian Ke- 6,4,0.1,0.2,0.496970
6,Pengujian Ke- 7,4,0.2,0.1,0.427273
7,Pengujian Ke- 8,4,0.2,0.2,0.387879


In [17]:
history.to_csv("history.csv", index=False)

### Deklarasi K, Alpha, dan Beta

### LDA

Dalam langkah berikutnya, saya menggunakan algoritma yang disebut Latent Dirichlet Allocation atau LDA untuk mengurangi dimensi data teks. LDA adalah algoritma yang membantu saya mengidentifikasi topik-topik utama yang muncul dalam dokumen-dokumen saya. Hasil dari ini adalah representasi dokumen-dalam-topik, yang artinya kita menggambarkan setiap dokumen sebagai kombinasi dari topik-topik yang ada.

Kami menerapkan LDA pada data pelatihan, yang menghasilkan matriks yang disebut `lda_x_train`. Matriks ini berisi representasi dokumen-dalam-topik untuk data pelatihan. Ini adalah cara kita menggambarkan hubungan antara dokumen dan topik dalam data pelatihan.

Kemudian, saya menggunakan model LDA yang sudah kita latih pada data pelatihan untuk mengubah data pengujian menjadi representasi dokumen-dalam-topik yang saya sebut sebagai `lda_x_test`. Ini membantu saya dalam memahami bagaimana dokumen-dokumen pengujian terkait dengan topik-topik yang telah saya identifikasi sebelumnya.

Dengan mengubah data teks ke dalam representasi topik-topik ini, saya memungkinkan penggunaan informasi ini dalam analisis lebih lanjut atau dalam pemodelan klasifikasi. Ini adalah salah satu cara di mana saya dapat mengolah dan memahami data teks dengan lebih baik."

In [18]:
lda = best_param['lda']
lda_x_train = best_param['lda_Xtrain']
lda_x_test = best_param['lda_Xtest']

### Tampilan Hasil Reduksi Dimensi

In [19]:
topik_columns = [f"Topik {i}" for i in range(1, best_param['k']+1)]
dokumen = final_tfidf['Dokumen']
output_proporsi_TD = pd.DataFrame(lda_x_train, columns=topik_columns)
output_proporsi_TD.insert(0,'Dokumen', dokumen)
output_proporsi_TD.insert(len(output_proporsi_TD.columns),'Kategori', final_tfidf['Kategori'])
output_proporsi_TD

,Dokumen,Topik 1,Topik 2,Topik 3,Kategori
0,jember beritajatimcom persema malang dihajar t...,0.975447,0.012278,0.012275,Sport
1,jakarta beritajatimcom persis solo dipimpin sa...,0.012163,0.975680,0.012157,Sport
2,jombang beritajatimcom orang jombang positif t...,0.010404,0.979196,0.010400,Pendidikan
3,sidoarjo beritajatimcom bupati sidoarjo h ahma...,0.978213,0.010895,0.010892,Politik
4,lamongan beritajatimcom pramuka lamongan mengg...,0.012626,0.974752,0.012623,Pendidikan
...,...,...,...,...,...
765,jakarta beritajatimcom tim jawa timur jatim me...,0.010796,0.978411,0.010793,Sport
766,surabaya beritajatimcom tim kampanye daerah tk...,0.979504,0.010249,0.010247,Politik
767,malang beritajatimcom universitas brawijaya ub...,0.012661,0.974679,0.012660,Pendidikan
768,surabaya beritajatimcom penonton piala dunia u...,0.012301,0.975401,0.012298,Sport


### Save Data hasil reduksi dimensi

In [20]:
output_proporsi_TD.to_csv('reduksi dimensi.csv', index=False)

### Tampilan proporsi kata di tiap topik

In [21]:
# Output distribusi kata pada topik
distribusi_kata_topik = pd.DataFrame(lda.components_)
distribusi_kata_topik

,0,1,2,3,4,5,6,7,8,9,...,30517,30518,30519,30520,30521,30522,30523,30524,30525,30526
0,0.255808,0.200000,0.200000,0.239401,0.2,0.200000,0.403769,0.200000,0.262487,0.2,...,0.200000,0.200000,0.200000,0.200000,0.405090,0.200000,0.326613,0.2,0.2,0.20000
1,0.200000,0.307846,0.253923,0.489376,0.2,0.386354,0.262244,0.402862,0.200000,0.2,...,0.264083,0.243335,0.291328,0.320986,0.499392,0.305657,0.292414,0.2,0.2,0.26144
2,0.210927,0.200000,0.200000,0.200000,0.2,0.200000,0.200000,0.200000,0.200000,0.2,...,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.2,0.2,0.20000


### Model Naive Bayes With LDA

In [22]:
# Membuat model Naive Bayes
model = best_param['model']

# Melakukan prediksi pada data pengujian
y_pred = model.predict(lda_x_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

# Menampilkan laporan klasifikasi
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

# Menampilkan matriks kebingungan
confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matriks:")
print(confusion)

Akurasi: 0.6212121212121212
Laporan Klasifikasi:
              precision    recall  f1-score   support

  Pendidikan       0.55      0.79      0.65       117
     Politik       0.00      0.00      0.00        84
       Sport       0.69      0.87      0.77       129

    accuracy                           0.62       330
   macro avg       0.41      0.55      0.47       330
weighted avg       0.47      0.62      0.53       330

Confusion Matriks:
[[ 93   0  24]
 [ 58   0  26]
 [ 17   0 112]]


### Predict

In [23]:
data = ["Penelitian ini menggabungkan konsep kecerdasan buatan dengan algoritma penjadwalan dalam upaya meningkatkan efisiensi produksi dalam lingkungan manufaktur. Kami memperkenalkan pendekatan yang memanfaatkan kecerdasan komputasional, yaitu algoritma optimasi berbasis swarm intelligence, seperti algoritma PSO (Particle Swarm Optimization) dan algoritma ACO (Ant Colony Optimization). Tujuan utama penelitian ini adalah untuk mengoptimalkan jadwal produksi dengan meminimalkan waktu produksi dan biaya, sambil mempertimbangkan berbagai kendala produksi seperti kapasitas mesin, waktu pemrosesan, dan persyaratan bahan baku. Melalui eksperimen dan simulasi, kami membandingkan hasil dari algoritma swarm intelligence dengan pendekatan konvensional. Hasilnya menunjukkan bahwa algoritma PSO dan ACO dapat menghasilkan jadwal produksi yang lebih efisien, dengan waktu produksi yang lebih pendek dan biaya yang lebih rendah. Selain itu, algoritma ini mampu beradaptasi dengan perubahan dalam lingkungan produksi dan menghasilkan jadwal yang optimal bahkan dalam situasi yang kompleks. Penelitian ini menunjukkan potensi besar dari penggunaan kecerdasan komputasional dalam meningkatkan efisiensi dan produktivitas dalam industri manufaktur. Hasil penelitian ini dapat digunakan sebagai dasar untuk mengembangkan sistem penjadwalan cerdas yang dapat diterapkan dalam berbagai industri."]
a = tfidf_vectorizer.transform(data).toarray()
b = lda.transform(a)
model.predict(b)

array(['Pendidikan'], dtype='<U10')

### Save Model

In [24]:
joblib.dump(lda, "lda.pkl")
joblib.dump(model, "naive bayes.pkl")

['naive bayes.pkl']

## Modeling Without LDA

### Training Model dengan Dataset Asli

In [25]:
# Membuat model Naive Bayes
nb = GaussianNB()

# Melatih model nbada data pelatihan
nb.fit(X_train, y_train)

# Melakukan prediksi pada data pengujian
y_pred_nb = nb.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred_nb)
print("Akurasi:", accuracy)

# Menampilkan laporan klasifikasi
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred_nb))

# Menampilkan matriks kebingungan
confusion = confusion_matrix(y_test, y_pred_nb)
print("Confusion Matriks:")
print(confusion)

Akurasi: 0.8848484848484849
Laporan Klasifikasi:
              precision    recall  f1-score   support

  Pendidikan       0.82      0.91      0.87       117
     Politik       0.84      0.79      0.81        84
       Sport       0.98      0.92      0.95       129

    accuracy                           0.88       330
   macro avg       0.88      0.87      0.88       330
weighted avg       0.89      0.88      0.89       330

Confusion Matriks:
[[107   9   1]
 [ 17  66   1]
 [  6   4 119]]


### Predict

In [26]:
data = ["Penelitian ini menggabungkan konsep kecerdasan buatan dengan algoritma penjadwalan dalam upaya meningkatkan efisiensi produksi dalam lingkungan manufaktur. Kami memperkenalkan pendekatan yang memanfaatkan kecerdasan komputasional, yaitu algoritma optimasi berbasis swarm intelligence, seperti algoritma PSO (Particle Swarm Optimization) dan algoritma ACO (Ant Colony Optimization). Tujuan utama penelitian ini adalah untuk mengoptimalkan jadwal produksi dengan meminimalkan waktu produksi dan biaya, sambil mempertimbangkan berbagai kendala produksi seperti kapasitas mesin, waktu pemrosesan, dan persyaratan bahan baku. Melalui eksperimen dan simulasi, kami membandingkan hasil dari algoritma swarm intelligence dengan pendekatan konvensional. Hasilnya menunjukkan bahwa algoritma PSO dan ACO dapat menghasilkan jadwal produksi yang lebih efisien, dengan waktu produksi yang lebih pendek dan biaya yang lebih rendah. Selain itu, algoritma ini mampu beradaptasi dengan perubahan dalam lingkungan produksi dan menghasilkan jadwal yang optimal bahkan dalam situasi yang kompleks. Penelitian ini menunjukkan potensi besar dari penggunaan kecerdasan komputasional dalam meningkatkan efisiensi dan produktivitas dalam industri manufaktur. Hasil penelitian ini dapat digunakan sebagai dasar untuk mengembangkan sistem penjadwalan cerdas yang dapat diterapkan dalam berbagai industri."]
tfidf_matrix = tfidf_vectorizer.transform(data).toarray()
nb.predict(tfidf_matrix)

array(['Pendidikan'], dtype='<U10')

### Save Model

In [27]:
joblib.dump(nb, "Naive Bayes (Asli).pkl")

['Naive Bayes (Asli).pkl']

## Save Vectorizer

In [28]:
joblib.dump(tfidf_vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']